In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms

import time
import copy
import numpy as np

from rs_attack import RSAttack

from torchvision import datasets
from torch.utils.data import DataLoader, ConcatDataset

In [ ]:
import logging

logger = logging.getLogger(__name__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.random.manual_seed(0)
torch.cuda.random.manual_seed(0)
np.random.seed(0)

In [ ]:
BATCH_SIZE = 128

# Data Preparation

In [ ]:
temp_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())

temp_loader = DataLoader(
        dataset=temp_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
data = next(iter(temp_loader))
mean = data[0].mean()
std = data[0].std()

transformers = transforms.Compose([
        transforms.RandomAffine(degrees = 30),
        transforms.RandomPerspective(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)])

train_dataset = datasets.MNIST(
        root='data', train=True, download=True, transform=transforms.ToTensor())

train_dataset_transformed = datasets.MNIST(
        root='data', train=True, download=True, transform=transformers)

test_dataset = datasets.MNIST(
        root='data', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
final_dataset = ConcatDataset([train_dataset, train_dataset_transformed])

In [ ]:
train_subset, valid_subset = torch.utils.data.random_split(
        final_dataset, [100_000, 20_000], generator=torch.Generator())

In [ ]:
train_loader = DataLoader(
        dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True)

valid_loader = DataLoader(
        dataset=valid_subset, batch_size=BATCH_SIZE, shuffle=False)

test_loader = DataLoader(
        dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
dataloaders_dict = {'train': train_loader, 'val': valid_loader}
dataset_sizes_dict = {'train': len(train_subset), 'val': len(valid_subset)}

# Definitions

In [ ]:
def train_model(model, criterion, optimizer, scheduler, attack, dataloaders, dataset_sizes, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:

            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                if attack:
                    inputs = attack(inputs)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, predictions = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predictions == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            logger.info('epoch:{} phase:{}'.format(epoch + 1, phase))
            logger.info(
                '{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since

    logger.info(
        'Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    logger.info(
        'Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)

    return model

In [ ]:
resnet = models.resnet18(weights='ResNet18_Weights.DEFAULT')
epochs = 20

resnet.conv1 = \
        nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet.fc = \
        nn.Linear(resnet.fc.in_features, 10)

resnet = resnet.to(device)

criterion_md = nn.CrossEntropyLoss()

optimizer_md = optim.Adam(resnet.parameters(), lr=0.001)

lr_scheduler_md = lr_scheduler.StepLR(optimizer_md, step_size=5, gamma=0.1)

In [ ]:
adversary = RSAttack(resnet, norm='L0', eps=20, verbose=True, n_queries=5_000, loss='ce')

In [ ]:
# train model
model_ft = train_model(
        resnet, criterion_md, optimizer_md, lr_scheduler_md, adversary.perturb,
        dataloaders_dict, dataset_sizes_dict, num_epochs=epochs)

# Prediction

In [ ]:
def predict(model, dataloader):

    test_pred = torch.LongTensor()
    with torch.no_grad():

        for images in dataloader:

            images = torch.autograd.Variable(images[0])
            if torch.cuda.is_available():
                images = images.to(device)

            outputs = model(images)
            predicted = outputs.cpu().data.max(1, keepdim=True)[1]
            test_pred = torch.cat((test_pred, predicted), dim=0)

    return test_pred

In [ ]:
results = predict(resnet, test_loader)